In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
from qdrant_client.http import models

from qdrant_client import QdrantClient
from qdrant_client.http.models import CollectionDescription, VectorParams, Distance
import sqlite3

client = QdrantClient(host="sprakbankdb1.lx.nb.no", port=6333)


In [49]:
import re
from IPython.display import HTML

def display_finds(r, num_rows, num_columns, width=500):
    """A list of urls in r is displayed in a grid layout with specified number of rows and columns."""
    base = "https://www.nb.no/items/"
    # Initialize the rows list which will contain HTML string for each row
    rows_html = []
    # Calculate total number of items to display, based on the specified rows and columns
    total_items = num_rows * num_columns
    # Ensure we don't try to display more items than we have
    r = r[:total_items]
    
    # Split the list into rows with the specified number of columns
    for row_start in range(0, len(r), num_columns):
        row_items = r[row_start:row_start+num_columns]
        # For each row, create a list of cell HTML strings
        cells_html = []
        for i, item in enumerate(row_items):
            urnstring = re.findall("URN[^/]*", item)[0]
            prefix, doctyp, urn, page = urnstring.split('_')
            cell_html = f"<td><a href='{base}{prefix}_{doctyp}_{urn}?page={int(page) + 1}' target='_'><img src='{item}' width={width}></a>{row_start+i}</td>"
            cells_html.append(cell_html)
        # Join the cell HTML strings into a row and add it to the rows list
        rows_html.append(f"<tr>{' '.join(cells_html)}</tr>")

    # Join all rows into the final HTML table
    html_table = f"<table>{' '.join(rows_html)}</table>"
    return HTML(f"""<html><head></head><body>{html_table}</body></html>""")


In [2]:
import numpy as np

In [33]:
collection_name = "images_1900_cos"

In [34]:
db = "../Bildevektor/all_images.db"

In [11]:
def query(db, sql, params=()):
    with sqlite3.connect(db) as con:
        r = con.execute(sql, params).fetchall()
    return r

In [12]:

def convert_vector(binary_vector):
    # Assuming binary_vector is a blob from SQLite, convert it to a list
    return np.frombuffer(binary_vector, dtype=np.float32).tolist()

In [ ]:
res = query(db, "select * from metadata order by random() limit 1 ")

In [65]:
import re

# Example strings
urls = [
    "https://www.nb.no/items/c04cb4454b7a9e615c68f6c50fc3dab4?page=0",
    "https://urn.nb.no/URN:NBN:no-nb_digifoto_AE0000025292_0053_F_01"
]

# Regex pattern

def fetch_iif(url):
    pattern = r"([a-fA-F0-9]{32})|(URN:[^?&\s]+)"
    matches = re.findall(pattern, url)
    match = ""
    if matches != []:
        match = matches[0]
    return match


In [ ]:
query_vector = convert_vector(res[0][2])

hits = client.search(
   collection_name = collection_name,
   query_vector = query_vector,
   limit = 20  # 
)

display_finds([x.payload['url'] for x in hits], 10, 4)

In [ ]:
for x in hits:
    print(x.payload['url'], x.score)
    

In [41]:
try:
    count_response = client.count(collection_name=collection_name)
    total_points = count_response.count
    print(f"Total points in collection '{collection_name}': {total_points}")
except Exception as e:
    print(f"An error occurred: {e}")

Total points in collection 'images_1900_cos': 422589


In [1]:
collections = client.list_collections()

# Print the names of the collections
for collection in collections.result.collections:
    print(collection.name)

NameError: name 'client' is not defined